# Chap05 - 순환 신경망(RNN)

단순한 **피드포워드(feedforward)** 신경망에서는 시계열 데이터의 성질(패턴)을 충분히 학습할 수 없다. 그 이유는 피드포워드가 한 방향으로만 신호가 전달되는 단방향 신경망이기 때문이다. 이를 해결하고자 **순환 신경망**(RNN, Recurrent Neural Network)이 등장하게 된다.

## 5.1 확률과 언어 모델

### 5.1.1 Word2Vec을 확률 관점에서 바라보다

Word2Vec의 CBOW 모델은 아래의 그림과 같이 `window_size=1`로 했을 때 맥락(context) $w_{t-1}$과 $w_{t+1}$로 부터 타깃(중앙) $w_t$를 예측하는 모델이다.



<img src="./images/cbow01.png" width="75%" height="75%" />

$w_{t-1}$과 $w_{t+1}$이 주어졌을 때 타깃이 $w_t$가 될 확률을 수식으로 나타내면 다음과 같다.

$$
P \left(w_t | w_{t-1}, w_{t+1}\right)
$$

CBOW 모델은 위의 사후확률을 모델링한다. 이 사후확률은 '$w_{t-1}$과 $w_{t+1}$이 주어졌을 때 $w_t$가 일어날 확률'을 뜻한다.

이번에는 `window_size`를 좌우 대칭으로 적용하지 말고, 왼쪽으로 설정하여 왼쪽의 맥락(context)으로 한정해 보도록 하자. 

<img src="./images/cbow02.png" width="75%" height="75%" />

$$
P(w_t | w_{t-2}, w_{t-1})
$$

위의 식에 Negative Log Likelihood를 적용하면 아래와 같이 손실함수를 정의할 수 있다. 

$$
L = -\log{P(w_t | w_{t-2}, w_{t-1})}
$$

이처럼 CBOW 모델을 학습시키는 본래 목적은 맥락으로부터 타깃을 정확하게 추측하는 것이다. 이렇게 학습을 시키게 되면, 그의 부산물로 단어의 의미가 인코딩된 '단어의 분산 표현'을 얻을 수 있다.

### 5.1.2 언어 모델

**언어 모델**(Language Model)은 단어의 나열에 확률을 부여한다. 특정한 단어의 시퀀스에 대해서, 그 시퀀스가 일어날 가능성이 어느 정도인지(얼마나 자연스러운 단어 순서인지)를 확률로 평가한다. 

또한, 언어 모델은 새로운 문장을 생성하는 용도로 이용할 수 있다. 언어 모델은 단어 순서의 자연스러움을 확률적으로 평가할 수 있기 때문에, 그 확률분포에 따라 다음으로 적합한 단어를 **샘플링**할 수 있기 때문이다.

이러한 언어 모델을 수식으로 살펴보면 $w_1 , \dots , w_m$이라는 $m$개의 단어로 된 문장이 있을 때, 단어가 $w_1, \dots, w_m$이라는 순서로 출현할 확률을 $P(w_1, \dots, w_m)$로 나타낼 수 있다. 이 확률은 여러 사건이 동시에 일어날 확률이므로 **동시 확률(Joint Probability, 또는 결합확률)**이라고 한다.

$$
\begin{align*}
P(w_1, \dots, w_m) &= P(w_m | w_1, \dots, w_{m-1}) P(w_{m-1} | w_1, \dots, w_{m-2}) \\ 
&\cdots P(w_3 | w_1, w_2) P(w_2 | w_1) P(w_1) \\
&= \prod_{t=1}^{m}{P(w_t | w_1, \dots, w_{t-1})}
\end{align*}
$$

위의 식은 확률의 **곱셈정리**로부터 유도할 수 있다. 확률의 곱셈정리는 다음과 같은 식으로 표현된다.

$$
P(A, B) = P(A|B)P(B)
$$

이 정리가 의미하는 것은 '$A$와 $B$가 모두 일어날 확률 $P(A, B)$'는 '$B$가 일어날 확률 $P(B)$'와 '$B$가 일어난 후 $A$가 일어날 확률 $P(A|B)$'를 곱한 값과 같다는 것이다.

> $P(A,B)$를 $P(A,B) = P(B|A)P(A)$ 처럼 분해할 수도 있는데, $A$와 $B$ 중 어느것을 **사후 확률(posterior probability)**의 조건으로 할지에 따라 2가지 표현 방법이 존재한다.

다시 언어 모델의 수식으로 돌아오면, 결합확률 $P(w_1, \dots, w_m)$은 사후확률의 총 곱인 $\prod{P(w_t|w_1, \dots, w_m)}$ 으로 나타낼 수 있다. 여기서 사후확률은 아래의 그림고 같이 타깃 단어보다 **왼쪽**에 있는 모든 단어를 맥락(조건)으로 했을 때의 확률이 된다.

<img src="./images/lm.png" width="75%" height="75%" />

위의 그림에서 $P(w_t | w_1, \dots, w_{t-1})$을 나타내는 모델은 **조건부 언어 모델**(Conditional Language Model)이라고 하는데, 보통 이를 **언어 모델**이라고 부른다.

### 5.1.3 CBOW 모델을 언어 모델로?

Word2Vec의 CBOW 모델을 언어 모델에 적용하기 위해서는 맥락(context)의 크기를 특정 값으로 한정하여 근사적으로 나타낼 수 있다. 맥락의 크기를 특정 값($n$)으로 한정하여 근사적으로 나타낼 수 있다. 

$$
P(w_1, \dots, w_m) = \prod_{t=1}^{m}{P(w_t|w_1, \dots, w_{t-1})} \approx \prod_{t=1}^{m}{P(w_t | w_{t-n}, \dots, w_{t-1})}
$$

위의 식처럼 맥락의 크기를 전체 단어의 개수가 아닌 $n$으로 설정하여 언어모델을 근사시킬 수 있다. 이러한 방법은 **마르코프 연쇄(Markov Chain)**을 따르는데, 마르코프 연쇄란 미래의 상태가 현재 상태에만 의존해 결정 된다는 것을 말한다. 위의 식에서는 $w_t$의 단어는 $w_{t-1}$에서 부터 $w_{t-n}$의 $n$개의 단어에만 의존하는 Markov Chain이라 할 수 있다.

하지만, CBOW(Continuous Bag-Of-Words) 모델은 모델이 의미하는 바와 같이, 맥락안의 단어의 순서는 고려하지 않는 문제가 있다. 맥락 단어의 순서를 고려하기 위해 아래의 그림과 같이 맥락의 단어 벡터를 은닉층에서 **연결**(concatenate)하는 방식인 NNLM(Neural Network Langauge Model) 모델이 있다. NNLM은 Bengio et.al(2003)이 제안한 방법으로 신경망을 이용한 언어 모델을 제안하였다.

![](./images/nnlm.png)

하지만 NNLM은 맥락의 크기에 비례하여 은닉층에서 concatenate 해주는 부분도 증가하기 때문에 학습시킬 가중치 매개변수 또한 증가하게 되는 문제가 있다.

> word2vec의 CBOW와 skip-gram 모델의 목적과 NNLM의 목적은 엄연히 다르다 할 수 있는데, CBOW와 Skip-gram 모델은 단어의 분산 표현(distributed representation)을 얻는 것에 목적을 두는 반면, NNLM은 언어 모델(Language Model)을 목적으로 한다.

## 5.2 RNN이란

### 5.2.1 순환하는 신경망

'순환한다(Recurrent)'는 의미는 '반복해서 되돌아감'을 의미하며, 어느 한 지점에서 시작해 시간을 지나 다시 원래 장소로 되돌아 오는 과정을 반복하는 것을 의미한다. 순환하기 위해서는 '닫힌 경로'가 필요하다.

RNN(Recurrent Neural Network)의 특징은 순환하는 경로(닫힌 경로)가 있다는 것이다. 이러한 순환 경로를 따라 데이터는 끊임없이 순환할 수 있고, 과거의 정보를 기억하는 동시에 최신 데이터로 갱신될 수 있다.

![](./images/rnn01.png)

위의 그림에서는 $\mathbf{x}_{t}$를 입력 받는데, $t$는 시각을 뜻한다. 시계열 데이터 $(\mathbf{x}_{0}, \mathbf{x}_{1}, \dots, \mathbf{x}_{t}, \dots)$가 RNN 계층에 입력되고, 그 입력에 대응하여 $(\mathbf{h}_{0}, \mathbf{h}_{1}, \dots, \mathbf{h}_{t}, \dots)$ 가 출력된다.

각 시각에 입력되는 $\mathbf{x}_{t}$는 벡터라고 가정한다. 문장(단어 순서)을 다루는 경우를 예로 든다면 각 단어의 분산 표현(단어 벡터)이 $\mathbf{x}_{t}$가 되며, 이 분산 표현이 순서대로 하나씩 RNN 계층에 입력된다. 위의 그림에서 출력이 2개로 '분기'가 되는 것을 알 수 있는데, 출력이 2개로 복제되어 그 중 하나가 다시 자신의 입력이 된다.

### 5.2.2 순환 구조 펼치기

![](./images/rnn02.png)

각 시각의 RNN 계층은 그 계층으로의 입력($\mathbf{x}_{t}$)과 1개 전의 RNN 계층으로 부터의 출력($\mathbf{h}_{t-1}$)을 받는다. 그런 다음, 이 두 정보를 바탕으로 현 시각의 출력을 계산한다.

$$
\mathbf{h}_{t} = \tanh{\left( \mathbf{h}_{t-1} \mathbf{W}_{\mathbf{h}} + \mathbf{x}_{t} \mathbf{W}_{\mathbf{x}} + \mathbf{b} \right)}
$$

$\mathbf{W}_{\mathbf{x}}$은 입력 $\mathbf{x}$를 출력 $\mathbf{h}$로 변환하기 위한 가중치이고, $\mathbf{W}_{\mathbf{h}}$는 출력 $\mathbf{h}_{t-1}$를 다음 시각의 출력으로 변환하기 위한 가중치이다. 또한 편향 $\mathbf{b}$도 있다. 

위의 식에서 행렬의 곱을 계산하고 그 합을 $\tanh$ 함수를 이용해 변환한 결과가 $\mathbf{h}_{t}$가 된다. 현재의 출력($\mathbf{h}_{t}$)은 한 시각 이전 출력($\mathbf{h}_{t-1}$)에 기초해 계산된다. RNN은 $\mathbf{h}$라는 **'상태'**(state)를 가지고 있다고 해석할 수 있다.

### 5.2.3 BPTT

RNN의 오차역전파법은 아래의 그림과 같이 RNN 계층을 시간 순으로 펼쳐서 나타낼 수 있다. 이를 '시간 방향으로 펼친 신경망의 오차역전파법'이란 뜻으로 **BPTT**(BackPropagation Through Time)이라고 한다.

![](./images/bptt01.png)

BPTT의 문제는 처음부터 끝까지 역전파하기 때문에 시간 크기(타임 스텝)가 커질수록 계산량이 많아질 뿐만아니라 역전파 시의 기울기가 불안정(vanishing & exploding gradient)해지는 문제가 있다.

### 5.2.4 Truncated BPTT

큰 시계열 데이터를 취급할 때는 역전파를 수행할 때 신경망 연결을 적당한 길이로 **'절단'**한다. 이러한 방법으로 역전파를 수행하는 것을 **Truncated BPTT**라고 한다.

Truncated BPTT에서는 신경망의 연결을 끊지만, '역전파'의 연결만 끊어야하고 순전파의 연결은 **반드시** 그대로 유지해야 한다. 그렇기 때문에 순전파의 흐름은 끊어지지 않고 전파되며, 역전파의 연결은 적당한 길이로 절단하여, 잘라낸 신경망 단위로 학습을 수행한다.

![](./images/bptt02.png)

위의 그림에서 확인할 수 있듯이, 역전파의 연결은 끊기지만 순전파의 연결은 끊기지 않는다. RNN을 학습시킬 때는 순전파가 연결되어 있다는 점을 고려해 데이터를 **'순서대로(sequential)'** 입력해야한다. 

> RNN을 제외한 다른 신경망에서는 미니배치를 통해 데이터를 주입할 때 랜덤하게 입력했지만, RNN에서는 데이터를 순서대로 입력해줘야 한다. 

![](./images/bptt03.png)

### 5.2.5 Truncated BPTT의 미니배치 학습

위에서 살펴본 Truncated BPTT의 학습처리순서는 미니배치를 고려하지 않은 즉, 미니배치가 1일 때에 해당하는 학습 방법이었다. 미니배치 수가 2 이상인 배치를 학습하기 위해서는 데이터를 입력하는 시작위치를 각 미니배치의 시작위치로 **'옮겨줘야'**한다.

길이가 `1,000`인 시계열 데이터에 대해 길이를 10개 단위로 잘라 Truncated BPTT로 학습하는 경우를 예로 들어보자. 이때, 미니배치의 수를 2개로 구성한다면 아래의 그림과 같이 구성할 수 있다. 

- 첫 번째 미니배치 때는 처음 부터 순서대로 데이터를 입력하고,
- 두 번째 미니배치 때는 `500`번 째 데이터를 시작위치로 정하고, 그 위치부터 다시 순서대로 데이터를 입력해 준다.

![](./images/bptt04.png)

## 5.3 RNN 구현

아래의 그림과 같이, $(\mathbf{x}_{0}, \mathbf{x}_{1}, \dots, \mathbf{x}_{T-1})$을 묶은 $\mathbf{xs}$를 입력하면 $(\mathbf{h}_{0}, \mathbf{h}_{1}, \dots, \mathbf{h}_{T-1})$을 묶은 $\mathbf{hs}$를 출력하는 단일 계층인 `TimeRNN`을 구현한다.

![](./images/timernn.png)

### 5.3.1 RNN 계층 구현

RNN의 순전파 식은 다음과 같다.

$$
\mathbf{h}_{t} = \tanh{\left( \mathbf{h}_{t-1} \mathbf{W}_{\mathbf{h}} + \mathbf{x}_{t} \mathbf{W}_{\mathbf{x}} + \mathbf{b} \right)}
$$

위의 식을 미니배치 사이즈 $N$개에 대해 Matrix의 `shape`를 맞춰주면 다음과 같다.

$$
\underbrace{\mathbf{h}_{t-1}}_{N \times H} \cdot \underbrace{\mathbf{W}_{\mathbf{h}}}_{H \times H} + \underbrace{\mathbf{x}_{t}}_{N \times D} \cdot \underbrace{\mathbf{W}_{\mathbf{x}}}_{D \times H} = \underbrace{\mathbf{h}_{t}}_{N \times H}
$$

![](./images/rnn_cell.png)

In [1]:
import sys
sys.path.append('..')
from common.np import *
from common.layers import *
from common.functions import sigmoid


class RNN:
    def __init__(self, Wx, Wh, b):
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.cache = None  # 역전파에 사용할 중간 데이터

    def forward(self, x, h, h_prev):
        Wx, Wh, b = self.params
        t = np.matmul(h_prev, Wh) + np.matmul(x, Wx) + b
        h_next = np.tanh(t)

        self.cache = (x, h_prev, h_next)
        return h_next

    def backward(self, dh_next):
        Wx, Wh, b = self.params
        x, h_prev, h_next = self.cache

        dt = dh_next * (1 - h_next ** 2)  # tanh 미분
        db = np.sum(dt, axis=0)
        dWh = np.dot(h_prev.T, dt)  # shape: (H, N) x (N, H) = (H, H)
        dh_prev = np.dot(dt, Wh.T)  # shape: (N, H) x (H, H) = (N, H)
        dWx = np.dot(x.T, dt)  # shape: (D, N) x (N, H) = (D, H)
        dx = np.dot(dt, Wx.T)  # shape: (N, H) x (H, D) = (N, D)

        self.grads[0][...] = dWx
        self.grads[1][...] = dWh
        self.grads[2][...] = db

        return dx, dh_prev

### 5.3.2 Time RNN 계층 구현

